# Decoding with Machine Learning

This notebook is going to load data, with the feature estimation steps already precomputed. Therefore, you cannot reproduce the results. We take this as a tutorial in how to use the decoding toolbox inside the py_neuromodulation pypeline. We also will run a typical exploratory data analysis - EDA, and will show how the user can take it's preferred models and cross-validation methods to run them here.

(We suggest you first follow example_BIDS.py and then come back here)

Again we start with importing the necessary packages:

notebook: ML with presaved data (run py_nm for upper data), read precomputed features, run decoding for individual channels, grid points, all channels combined. Visualize decoding results. Add typical EDA, e.g. boxplot of a single feature movement vs rest, time plot, correlation matrix of features,

In [1]:
import os
import sys

# change root directory of the project
SCRIPT_DIR = os.path.dirname(os.path.abspath(''))
if os.path.basename(SCRIPT_DIR) == "py_neuromodulation":
    # this check is necessary, so we can also run the script from the root directory
    SCRIPT_DIR = os.path.join(SCRIPT_DIR, "examples")

sys.path.append(os.path.dirname(SCRIPT_DIR))

# Reload imports to get changes in other scripts
%load_ext autoreload
%autoreload 2

In [2]:
import py_neuromodulation as nm
from py_neuromodulation import (
    nm_analysis,
    nm_decode,
    nm_define_nmchannels,
    nm_IO,
    nm_plots,
)
from sklearn import metrics, model_selection
from skopt import space as skopt_space
from scipy import io
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Run here the feature estimation steps in the Berlin data set subject, then export it to example_decodingML

import os
import py_neuromodulation as nm
from py_neuromodulation import (
    nm_analysis,
    nm_decode,
    nm_define_nmchannels,
    nm_IO,
    nm_plots,
)

sub = "006"
ses = "EcogLfpMedOff02"
task = "SelfpacedRotationL"
acq = "StimOn"
run = 1
datatype = "ieeg"

# Define run name and access paths in the BIDS format.
RUN_NAME = f"sub-{sub}_ses-{ses}_task-{task}_acq-{acq}_run-{run}"

PATH_RUN = os.path.join(
    "/home/lauraflyra/Documents/BCCN/Lab_Rotation_DBS_Decoding/BIDS_data",
    f"sub-{sub}",
    f"ses-{ses}",
    datatype,
    RUN_NAME,
)
PATH_BIDS = "/home/lauraflyra/Documents/BCCN/Lab_Rotation_DBS_Decoding/BIDS_data"

# Provide a path for the output data.
PATH_OUT = os.path.join("/home/lauraflyra/Documents/BCCN/Lab_Rotation_DBS_Decoding/BIDS_data",
                        "derivatives")

(
    raw,
    data,
    sfreq,
    line_noise,
    coord_list,
    coord_names,
) = nm_IO.read_BIDS_data(
    PATH_RUN=PATH_RUN, BIDS_PATH=PATH_BIDS, datatype=datatype
)

Extracting parameters from /home/lauraflyra/Documents/BCCN/Lab_Rotation_DBS_Decoding/BIDS_data/sub-006/ses-EcogLfpMedOff02/ieeg/sub-006_ses-EcogLfpMedOff02_task-SelfpacedRotationL_acq-StimOn_run-1_ieeg.vhdr...
Setting channel info structure...
Reading channel info from /home/lauraflyra/Documents/BCCN/Lab_Rotation_DBS_Decoding/BIDS_data/sub-006/ses-EcogLfpMedOff02/ieeg/sub-006_ses-EcogLfpMedOff02_task-SelfpacedRotationL_acq-StimOn_run-1_channels.tsv.
Reading electrode coords from /home/lauraflyra/Documents/BCCN/Lab_Rotation_DBS_Decoding/BIDS_data/sub-006/ses-EcogLfpMedOff02/ieeg/sub-006_ses-EcogLfpMedOff02_space-MNI152NLin2009bAsym_electrodes.tsv.


In [5]:
raw

<RawBrainVision | sub-006_ses-EcogLfpMedOff02_task-SelfpacedRotationL_acq-StimOn_run-1_ieeg.eeg, 33 x 1869183 (467.3 s), ~47 kB, data not loaded>

In [8]:
nm_channels = nm_define_nmchannels.set_channels(
    ch_names=raw.ch_names,
    ch_types=raw.get_channel_types(),
    reference="default",
    bads=raw.info["bads"],
    new_names="default",
    used_types=("ecog", "dbs", "eeg", "emg"),
   target_keywords=("ACC_R_X_D2_TM",),
)

In [7]:
nm_channels

,name,rereference,used,target,type,status,new_name
0,LFP_R_01_STN_MT,LFP_R_08_STN_MT,1,0,dbs,good,LFP_R_01_STN_MT-LFP_R_08_STN_MT
1,LFP_R_02_STN_MT,LFP_R_01_STN_MT,0,0,dbs,bad,LFP_R_02_STN_MT-LFP_R_01_STN_MT
2,LFP_R_03_STN_MT,LFP_R_02_STN_MT,0,0,dbs,bad,LFP_R_03_STN_MT-LFP_R_02_STN_MT
3,LFP_R_04_STN_MT,LFP_R_03_STN_MT,0,0,dbs,bad,LFP_R_04_STN_MT-LFP_R_03_STN_MT
4,LFP_R_05_STN_MT,LFP_R_04_STN_MT,1,0,dbs,good,LFP_R_05_STN_MT-LFP_R_04_STN_MT
5,LFP_R_06_STN_MT,LFP_R_05_STN_MT,1,0,dbs,good,LFP_R_06_STN_MT-LFP_R_05_STN_MT
6,LFP_R_07_STN_MT,LFP_R_06_STN_MT,1,0,dbs,good,LFP_R_07_STN_MT-LFP_R_06_STN_MT
7,LFP_R_08_STN_MT,LFP_R_07_STN_MT,1,0,dbs,good,LFP_R_08_STN_MT-LFP_R_07_STN_MT
8,LFP_L_01_STN_MT,LFP_L_08_STN_MT,0,0,dbs,bad,LFP_L_01_STN_MT-LFP_L_08_STN_MT
9,LFP_L_02_STN_MT,LFP_L_01_STN_MT,0,0,dbs,bad,LFP_L_02_STN_MT-LFP_L_01_STN_MT
